In [118]:
import pandas as pd
import re
import numpy as np

In [119]:
df = pd.read_csv('../release/data.csv')
pd.set_option('display.max_columns', None)

# Приведение значений столбца 'windows' к числовым

In [120]:
df.windows.value_counts()

windows
Во двор                     13954
На улицу и двор              3172
На улицу                     2444
На достопримечательность      167
Name: count, dtype: int64

In [121]:
#Сделаем 3 стобца с 1-если да, 0 -если нет
def code_windows(data, feature):
    for i in data[feature].unique():
        data[i] = (data[feature] == i).astype(float)

code_windows(df, 'windows')

In [122]:
def code_windows_2(row):
    if row['На улицу и двор'] == 1:
        row['На улицу'] = 1
        row['Во двор'] = 1
    return row

df = df.apply(code_windows_2, axis=1)

In [123]:
df.drop(['windows', 'На улицу и двор'], axis=1, inplace=True)

In [124]:
df.rename(columns={'Во двор': 'courtyard_view', 'На достопримечательность': 'landmark_view', 'На улицу': 'street_view'}, inplace=True)

# Приведение значений из столбца 'bathroom' к числовым

Вынесем в отдельные признаки общее количество сан. узлов в квартире и их вид

In [125]:
#Добавления столбца с количеством санузлов:
def bathroom_count(row):
    counts = re.findall(r'\d+', row)
    return sum(map(int, counts))

# Применяем функцию к столбцу "bathroom"
df['bathroom_count'] = df['bathroom'].apply(bathroom_count)



In [126]:
#Добавление столбцов с видом санузлов (совмещенный и раздельный)
def bathroom_kind(row):
    if 'Совмещенный' in row['bathroom'] and 'Раздельный' in row['bathroom']:
        row['combined_bath'] = 1
        row['separate_bath'] = 1
    elif 'Совмещенный' in row['bathroom']:
        row['combined_bath'] = 1
    elif 'Раздельный' in row['bathroom']:
        row['separate_bath'] = 1

    return row

df['combined_bath'] = np.nan
df['separate_bath'] = np.nan

df = df.apply(bathroom_kind, axis=1)
df.loc[:, ['combined_bath', 'separate_bath']] = df[['combined_bath', 'separate_bath']].fillna(0)

In [127]:
df.drop('bathroom', axis=1, inplace=True)

# Приведение столбца 'сhildren/pets_allowed' к численным значениям

Сделаем 2 столбца с наличием/отсутствием соответствующих признаков

In [128]:
df['сhildren/pets_allowed'].unique()

array(['Можно с детьми, Можно с животными', 'Можно с детьми',
       'Можно с животными', 'Нельзя никого'], dtype=object)

In [129]:
def bathroom_kind(row):
    if 'Можно с детьми' in row['сhildren/pets_allowed'] and 'Можно с животными' in row['сhildren/pets_allowed']:
        row['pets_allowed'] = 1
        row['children_allowed'] = 1
    elif 'Можно с детьм' in row['сhildren/pets_allowed']:
        row['children_allowed'] = 1
    elif 'Можно с животными' in row['сhildren/pets_allowed']:
        row['pets_allowed'] = 1

    return row

df['pets_allowed'] = np.nan
df['children_allowed'] = np.nan

df = df.apply(bathroom_kind, axis=1)
df.loc[:, ['pets_allowed', 'children_allowed']] = df[['pets_allowed', 'children_allowed']].fillna(0)

In [130]:
df.drop('сhildren/pets_allowed', axis=1, inplace=True)

# Приведение столбца 'additionally' к численным значениям

Добавим признаки наличия мебели и наличия крупной бытовой техники

In [131]:
df.additionally.unique()

array(['Мебель в комнатах, Мебель на кухне, Ванна, Душевая кабина, Стиральная машина, Кондиционер, Посудомоечная машина, Телевизор, Холодильник, Интернет, Телефон',
       'Мебель в комнатах, Мебель на кухне, Ванна, Душевая кабина, Стиральная машина, Кондиционер, Посудомоечная машина, Телевизор, Холодильник, Интернет',
       'Мебель в комнатах, Мебель на кухне, Ванна, Душевая кабина, Стиральная машина, Посудомоечная машина, Телевизор, Холодильник, Интернет',
       'Мебель в комнатах, Мебель на кухне, Ванна, Душевая кабина, Стиральная машина, Кондиционер, Посудомоечная машина, Телевизор, Холодильник',
       'Мебель в комнатах, Мебель на кухне, Ванна, Стиральная машина, Посудомоечная машина, Телевизор, Холодильник, Интернет, Телефон',
       'Мебель в комнатах, Мебель на кухне, Ванна, Душевая кабина, Стиральная машина, Кондиционер, Посудомоечная машина, Холодильник, Интернет',
       'Мебель на кухне, Ванна, Стиральная машина, Кондиционер, Посудомоечная машина, Телевизор, Холодильник,

In [132]:
def code_additionally(row):
    addition_furn = 'мебель'
    addition_tech = ['техника', 'холодильник', 'стиральная', 'посудомоечная']
    
    row['furniture'] = 1.0 if addition_furn in row['additionally'].lower() else 0
    row['home_tech'] = 1.0 if any(word in row['additionally'].lower() for word in addition_tech) else 0

    return row


df['furniture'] = np.nan
df['home_tech'] = np.nan

df = df.apply(code_additionally, axis=1)

In [133]:
df.drop('additionally', axis=1, inplace=True)

In [137]:
vera_df = df.loc[:, 'courtyard_view': 'home_tech']

In [144]:
vera_df

,courtyard_view,landmark_view,street_view,bathroom_count,combined_bath,separate_bath,pets_allowed,children_allowed,furniture,home_tech
0,1.0,0.0,0.0,2,1.0,0.0,1.0,1.0,1.0,1.0
1,1.0,0.0,1.0,3,1.0,1.0,0.0,1.0,1.0,1.0
2,1.0,0.0,1.0,3,1.0,0.0,0.0,1.0,1.0,1.0
3,1.0,0.0,1.0,3,1.0,0.0,1.0,0.0,1.0,1.0
4,1.0,0.0,1.0,2,1.0,0.0,0.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...
19732,1.0,0.0,0.0,1,1.0,0.0,1.0,0.0,1.0,1.0
19733,1.0,0.0,0.0,1,1.0,0.0,1.0,0.0,1.0,1.0
19734,1.0,0.0,1.0,1,1.0,0.0,0.0,1.0,1.0,1.0
19735,1.0,0.0,0.0,2,1.0,1.0,0.0,1.0,1.0,1.0


In [143]:
# vera_df.to_csv('../notebooks/vera_df.csv')